# **Estimate Facade Greening Potenial**

In [ ]:
# Sometimes encoding needs to be specified because of a Google Colab bug
import locale

locale.getpreferredencoding = lambda: "UTF-8"
print(locale.getpreferredencoding())

## **Install dependencies**

Install the required packages. Restart the runtime and rerun the block below if it is requested

In [ ]:
%pip install Pillow==10.0.1 segment-geospatial groundingdino-py leafmap localtileserver geopandas ipython mercantile networkx numpy osmnx pandas Requests scipy Shapely tqdm vt2geojson area

## **Specify working directory**

Input your working directory path and choose whether you will work using Google Drive or locally

In [ ]:
working_dir = '/My Drive/FolderName'
work_on_drive = True

Mount your Google Drive

In [ ]:
from google.colab import drive
import os


if work_on_drive == True:
  drive_dir = '/content/drive'
  drive.mount(drive_dir)
  os.chdir(drive_dir + working_dir)
else:
  os.chdir(working_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Import the required modules**

In [ ]:
from modules.grid_processing import *
from modules.main_processing import *

from samgeo.text_sam import LangSAM
import leafmap
import json

## **Specify parameters**

Specify the values for the following input variables

In [ ]:
city_name = 'Diamantbuurt, Amsterdam'            # Choose a city (and neighborhood) (e.g. 'Amsterdam' or 'Diamantbuurt, Amsterdam')

access_token = 'MLY|XXXXXXXX|xxxxxxxxxxxxxxxx'   # Required Mapillary access token
distance = 50                                    # Sample point distance in meters
num_sample_images = float('inf')                 # Max number of points analyzed per processing area (bbox or named area)
begin = None                                     # Starting index of limited sample range (if any)
end = None                                       # Ending index of limited sample range (if any)
save_roads_points = True                         # Save roads and points geopackages
save_streetview = False                          # Save analyzed street view imagery

Draw a map on which a study area can be selected using the rectangle selection

If no selection is made, the city name will be used as study area

In [ ]:
m = leafmap.Map(center=[52.37223326302798, 4.899871868930255], zoom=14, height="800px")
m.add_basemap("SATELLITE")
m

In [ ]:
# Bounding box areas used for Amsterdam analysis

# Full area:
# bbox = [4.590912, 52.284962, 5.105896, 52.467724]

# First half (WEST):
# bbox = [4.590912, 52.284962, 4.848404, 52.467724]

# Second half (EAST):
# bbox = [4.848404, 52.284962, 5.105896, 52.467724]

# Center:
# bbox = [4.892006, 52.366795, 4.897199, 52.377459]

In [ ]:
# Save the user's bounding box (bbox) input
bbox = m.user_roi_bounds(decimals=8)
print(f"Bounding Box: {bbox}")

# Check if the user specified a bounding box selection
if bbox:
  city = f"{city_name}_BBOX"
  print(city)

  # Calculate area of the initial bbox
  x_neg = bbox[0]
  y_neg = bbox[1]
  x_pos = bbox[2]
  y_pos = bbox[3]

  bbox_area = rect_area(x_neg, y_neg, x_pos, y_pos)
  print(f"Area in Ha: {bbox_area}")

  # Split up area if it's bigger than 400 Ha per processing bbox
  max_area_ha = 400
  rectangles = split_rectangle(max_area_ha, x_neg, y_neg, x_pos, y_pos)

  # Create Shapely polygons from bboxes
  shapely_bboxes = []

  for i, rectangle in enumerate(rectangles):
    polygon = coords_to_shapely_polygon(rectangle[0], rectangle[1], rectangle[2], rectangle[3])
    shapely_bboxes.append(polygon)

  # Create gdf with Shapely polygons and same coordinate system as Amsterdam border
  gdf_bbox_polygons = gpd.GeoDataFrame(geometry=shapely_bboxes, crs='EPSG:4326')
  gdf_bbox_polygons = gdf_bbox_polygons.to_crs(epsg=28992)

  # Get Amsterdam boundaries
  municipal_boundaries = 'bestuurlijkegrenzen.gpkg'
  gdf_boundaries = gpd.read_file(municipal_boundaries, crs='EPSG:28992', layer='gemeenten')
  gdf_amsterdam = gdf_boundaries[gdf_boundaries['gemeentenaam'] == 'Amsterdam'].iloc[[0]]

  # Compute centroids for each polygon
  gdf_bbox_polygons['centroid'] = gdf_bbox_polygons['geometry'].centroid

  # Check if centroid is within Amsterdam polygon
  Amsterdam_polygon = gdf_amsterdam.at[50, 'geometry']
  save_rows = []

  for index, row in gdf_bbox_polygons.iterrows():
    centroid = row['centroid']

    if centroid.within(Amsterdam_polygon):
      save_rows.append(index)

  gdf_bbox_polygons_within = gdf_bbox_polygons.iloc[save_rows]

  # Drop centroid column so the gdf can be visualized
  gdf_bbox_polygons_within = gdf_bbox_polygons_within.drop(columns=['centroid'])

  # Update the rectangles list that contains processing bboxes
  filtered_rectangles = [rectangles[i] for i in save_rows]

  # Show the bboxes and Amsterdam border on the map
  m.add_gdf(gdf_amsterdam, layer_name='Amsterdam')
  m.add_gdf(gdf_bbox_polygons_within, layer_name='Bboxes')

  n_bboxes_total = len(shapely_bboxes)
  n_bboxes_inside = len(filtered_rectangles)
  print(f"Split the area into {n_bboxes_total} rectangles.\n{n_bboxes_inside} are inside the study area (see map)")

# If the user did not draw a bbox, the city name will be used as a location
else:
  bbox = False
  city = city_name

## **Calculate facade greening potential score**

*Note: if the process was stopped abruptly, check the temporary (temp) folders in* `results/MyCityName/` *and remove any images there before continuing*

In [ ]:
# Create folder structure
prepare_folders(city)

# Initialize SAM (segmentation model)
sam = LangSAM()

if bbox:
  # Store the merged results of each rectangle
  total_usable_ratios = []

  # Create a text file that remembers which bboxes are already processed
  processed_bboxes = os.path.join("results", city, "processed_bboxes.txt")

  # Initialize the file if it doesnt exist yet
  if not os.path.exists(processed_bboxes):
    print("Creating processed_bboxes.txt")
    with open(processed_bboxes, "w") as file:
      pass

  # Loop over each (processing) sub-bbox
  for i, sub_bbox in enumerate(filtered_rectangles):
    print(f"Processing area {i+1} of {n_bboxes_inside}")
    flag = False

    # Check if this sub_bbox was already processed
    with open(processed_bboxes, "r") as file:
      for line in file:
        # If so, retrieve the saved usable ratios
        if str(sub_bbox) in line:
          columns = line.split("\t")
          saved_ratios = json.loads(columns[1])
          total_usable_ratios.extend(saved_ratios)
          flag = True

    # If it's already processed, skip this sub-bbox
    if flag == True:
      print(f"Area {i+1} already processed. Continuing...")
      continue

    # Prepare the features
    features = create_features(city, sub_bbox, access_token, distance, num_sample_images, begin, end, save_roads_points, i)

    # If there are no roads, skip this sub-bbox
    if features.empty:
      print(f"No roads/features found inside bbox {i+1}. Continuing...")
      continue

    # Initialize SAM and calculate the usable wall ratios
    usable_ratios = calculate_usable_wall_ratios(features, city, sam, access_token, save_streetview, str(i))

    # Save the usable wall ratios
    for ratio in usable_ratios:
      total_usable_ratios.append(ratio)

    # Do not process this sub-bbox anymore if the program is interrupted
    with open(processed_bboxes, "a") as file:
      file.write(f"\n{str(sub_bbox)}\t{usable_ratios}")

    # Save the features as a geopackage file
    save_usable_wall_ratios(city, total_usable_ratios)

# If no bbox was used, process like normal
else:
  # Prepare the features
  features = create_features(city, bbox, access_token, distance, num_sample_images, begin, end, save_roads_points)

  # Initialize SAM and calculate the usable wall ratios
  total_usable_ratios = calculate_usable_wall_ratios(features, city, sam, access_token, save_streetview)

# Save the features as a geopackage file
save_usable_wall_ratios(city, total_usable_ratios)

print("Finished!")